# Imports

In [38]:
train_full_url = 'https://raw.githubusercontent.com/sidt-ai/data-science-competitions/main/machinehack/ode_to_code_2022/data/processed/train_full.csv'
train_reduced_url = 'https://raw.githubusercontent.com/sidt-ai/data-science-competitions/main/machinehack/ode_to_code_2022/data/processed/train_reduced.csv'
test_url = 'https://raw.githubusercontent.com/sidt-ai/data-science-competitions/main/machinehack/ode_to_code_2022/data/processed/test.csv'
sub_url = 'https://raw.githubusercontent.com/sidt-ai/data-science-competitions/main/machinehack/ode_to_code_2022/data/raw/sample_submission.csv'

In [39]:
%%capture
!python3 -m pip install --quiet --upgrade optuna
!python3 -m pip install --quiet --upgrade xgboost==1.5
!python3 -m pip install --quiet --upgrade catboost
!python3 -m pip install --quiet --upgrade lightgbm

In [40]:
%%capture
!python3 -m pip install --quiet --upgrade sklearn

In [41]:
import time
import gc
gc.enable()

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.set_option("precision", 4)

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="darkgrid")

from scipy.stats import mode
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split

import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier

import optuna
from optuna.samplers import TPESampler

SEED = 2311

In [42]:
import torch

if torch.cuda.is_available():
    GPU = True
else:
    GPU = False

In [43]:
train_full = pd.read_csv(train_full_url)
train_reduced = pd.read_csv(train_reduced_url)
test = pd.read_csv(test_url)
sub = pd.read_csv(sub_url)

In [44]:
target = ['season']
features = [f for f in train_full.columns if f not in ('season', 'fold')]
num_features = ['cap-diameter', 'stem-height', 'stem-width']
cat_features = [f for f in features if f not in num_features]

In [45]:
target_mapping = {0: 'a', 1: 's', 2: 'u', 3: 'w'}

# CatBoost

### Model 6 - train_full

In [20]:
cb_base = {
    'loss_function': 'MultiClass',
    'custom_metric': 'F1',
    'eval_metric': 'Accuracy',
    'iterations': 5000,
    'random_seed': SEED,
    'use_best_model': True,
    'classes_count': 4,
    'task_type': 'CPU',
    'cat_features': cat_features
}

if GPU:
  cb_base['task_type'] = 'GPU'

In [16]:
def cb_objective(trial, xtrain, ytrain, xval, yval, cb_base):
  
  cb_grid = {
    'learning_rate' :trial.suggest_loguniform('learning_rate', 0.01, 0.3),
    'depth': trial.suggest_int('depth', 1, 15),
    'l2_leaf_reg' :trial.suggest_loguniform('l2_leaf_reg', 1e-8, 20),
    'random_strength' : trial.suggest_loguniform('random_strength', 1, 50),
    'grow_policy':trial.suggest_categorical(
        'grow_policy', ['Lossguide','SymmetricTree']
    ),
    'max_bin': trial.suggest_int("max_bin", 20, 500),
    'min_data_in_leaf':trial.suggest_int('min_data_in_leaf', 1, 100),
    'bootstrap_type': trial.suggest_categorical(
        'bootstrap_type', ['Bayesian', 'Bernoulli']
    )
  }
              
  if cb_grid['grow_policy'] == 'Lossguide':
    cb_grid['max_leaves']:trial.suggest_int('max_leaves', 1, 100)
    
  if cb_grid["bootstrap_type"] == "Bayesian":
    cb_grid["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
  elif cb_grid["bootstrap_type"] == "Bernoulli":
    cb_grid["subsample"] = trial.suggest_float("subsample", 0.1, 1)

  model = CatBoostClassifier(**cb_base, **cb_grid)

  model.fit(
      xtrain, ytrain,
      eval_set=[(xval, yval)],
      verbose=100,
      early_stopping_rounds=100
  )

  val_predictions = model.predict(xval)
  return accuracy_score(yval, val_predictions)  

In [17]:
cbfull_study = optuna.create_study(direction='maximize', 
                            sampler=TPESampler(), 
                            study_name='MH_O2C_M6')

[I 2022-01-24 11:10:14,303] A new study created in memory with name: MH_O2C_M6


In [18]:
xtrain, xval, ytrain, yval = train_test_split(train_full[features], 
                                              train_full[target],
                                              test_size=0.2, shuffle=True,
                                              stratify=train_full[target],
                                              random_state=SEED)

In [ ]:
# %%capture
cbfull_study.optimize(
    lambda trial: cb_objective(trial, xtrain, ytrain, xval, yval, cb_base),
    n_trials=50
)

In [22]:
print(f'Best value (Accuracy): {cbfull_study.best_value:.5f}')

cbfull_best = cbfull_study.best_params
print('Best params:')
for key, value in cbfull_best.items():
    print(f'\t{key}: {value}')

Best value (Accuracy): 0.53216
Best params:
	learning_rate: 0.24912820430081778
	depth: 14
	l2_leaf_reg: 0.002167945616348965
	random_strength: 16.825720017427575
	grow_policy: Lossguide
	max_bin: 190
	min_data_in_leaf: 27
	bootstrap_type: Bernoulli
	subsample: 0.26524979251881575


In [26]:
N_SPLITS = 5

def custom_cross_val_predict(train, test, features, model):
  oof_preds = {}
  test_preds = []
  scores = []
    
  cv_start = time.time()
    
  for fold in range(N_SPLITS):
    print('-' * 40)
        
    xtrain = train[train.fold != fold].reset_index(drop=True)
    ytrain = xtrain.season

    xval = train[train.fold == fold].reset_index(drop=True)
    yval = xval.season
    val_idx = xval.index.tolist()
    
    fold_start = time.time()
        
    model.fit(
      xtrain[features], ytrain,
      eval_set=[(xval[features], yval)],
      early_stopping_rounds=100,
      verbose=100
    )
    
    val_preds = model.predict(xval[features])
    oof_preds.update(dict(zip(val_idx, val_preds)))
    acc = accuracy_score(yval, val_preds)
    scores.append(acc)
        
    fold_end = time.time()
        
    print(f'Fold #{fold}: Accuracy = {acc:.5f} \
    [Time: {fold_end - fold_start:.2f}s]')
        
    test_preds.append(model.predict(test[features]))
        
    cv_end = time.time()
  
  print(f'Average accuracy = {np.mean(scores):.5f} \
  with std. dev. = {np.std(scores):.5f}')
  
  print(f'[Total time: {cv_end - cv_start:.2f}s]')
    
  oof_preds = pd.DataFrame.from_dict(oof_preds, orient='index').reset_index()
  test_preds = mode(np.column_stack(test_preds), axis=1).mode
    
  return oof_preds, test_preds

In [27]:
model6 = CatBoostClassifier(**cb_base, **cbfull_best)

In [29]:
oof_preds6, test_preds6 = custom_cross_val_predict(train_full, test, features, model6)

----------------------------------------


Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 0.5121645	test: 0.5182456	best: 0.5182456 (0)	total: 38ms	remaining: 3m 10s
100:	learn: 0.5917013	test: 0.5155556	best: 0.5219883 (10)	total: 3.68s	remaining: 2m 58s
bestTest = 0.5219883041
bestIteration = 10
Shrink model to first 11 iterations.
Fold #0: Accuracy = 0.52199     [Time: 6.08s]
----------------------------------------
0:	learn: 0.5129540	test: 0.5112281	best: 0.5112281 (0)	total: 37.2ms	remaining: 3m 5s
100:	learn: 0.5941576	test: 0.5098246	best: 0.5184795 (9)	total: 3.52s	remaining: 2m 50s
bestTest = 0.5184795322
bestIteration = 9
Shrink model to first 10 iterations.
Fold #1: Accuracy = 0.51848     [Time: 5.80s]
----------------------------------------
0:	learn: 0.5149132	test: 0.5066667	best: 0.5066667 (0)	total: 35.1ms	remaining: 2m 55s
100:	learn: 0.5999766	test: 0.5134503	best: 0.5170760 (3)	total: 3.33s	remaining: 2m 41s
bestTest = 0.5170760234
bestIteration = 3
Shrink model to first 4 iterations.
Fold #2: Accuracy = 0.51708     [Time: 5.21s]
--------------

In [30]:
sub6 = pd.read_csv(sub_url)
sub6['season'] = test_preds6.ravel()
sub6['season'] = sub6['season'].map(target_mapping)
sub6.to_csv('sub6.csv', index=False)

!head sub6.csv

season
a
a
u
a
a
u
a
u
u


### Model 7 - train_reduced

In [31]:
cbred_study = optuna.create_study(direction='maximize', 
                            sampler=TPESampler(), 
                            study_name='MH_O2C_M7')

[I 2022-01-24 11:32:41,379] A new study created in memory with name: MH_O2C_M7


In [32]:
xtrain, xval, ytrain, yval = train_test_split(train_reduced[features], 
                                              train_reduced[target],
                                              test_size=0.2, shuffle=True,
                                              stratify=train_reduced[target],
                                              random_state=SEED)

In [33]:
%%capture
cbred_study.optimize(
    lambda trial: cb_objective(trial, xtrain, ytrain, xval, yval, cb_base),
    n_trials=50
)

0:	learn: 0.5257045	test: 0.5226445	best: 0.5226445 (0)	total: 32.9ms	remaining: 2m 44s
100:	learn: 0.5782786	test: 0.5184214	best: 0.5273045 (4)	total: 3.12s	remaining: 2m 31s
bestTest = 0.5273044998
bestIteration = 4
Shrink model to first 5 iterations.


[I 2022-01-24 11:32:46,280] Trial 0 finished with value: 0.527304499781564 and parameters: {'learning_rate': 0.08812288225265932, 'depth': 7, 'l2_leaf_reg': 4.878422636480148, 'random_strength': 1.1973944020005238, 'grow_policy': 'Lossguide', 'max_bin': 417, 'min_data_in_leaf': 20, 'bootstrap_type': 'Bernoulli', 'subsample': 0.9589565921664691}. Best is trial 0 with value: 0.527304499781564.


0:	learn: 0.4995995	test: 0.5006553	best: 0.5006553 (0)	total: 32.5ms	remaining: 2m 42s
100:	learn: 0.4879487	test: 0.4687637	best: 0.5026940 (1)	total: 3.08s	remaining: 2m 29s
bestTest = 0.502694044
bestIteration = 1
Shrink model to first 2 iterations.


[I 2022-01-24 11:32:50,913] Trial 1 finished with value: 0.5026940439784476 and parameters: {'learning_rate': 0.021492988592547762, 'depth': 8, 'l2_leaf_reg': 5.93540064674474e-08, 'random_strength': 21.608259468348287, 'grow_policy': 'SymmetricTree', 'max_bin': 352, 'min_data_in_leaf': 62, 'bootstrap_type': 'Bernoulli', 'subsample': 0.6880655280284255}. Best is trial 0 with value: 0.527304499781564.


0:	learn: 0.5172577	test: 0.5137615	best: 0.5137615 (0)	total: 50.6ms	remaining: 4m 13s
100:	learn: 0.5374281	test: 0.5300714	best: 0.5306539 (91)	total: 3.32s	remaining: 2m 40s
bestTest = 0.5306538518
bestIteration = 91
Shrink model to first 92 iterations.


...

[I 2022-01-24 11:41:57,147] Trial 47 finished with value: 0.5382262996941896 and parameters: {'learning_rate': 0.29547218456097296, 'depth': 6, 'l2_leaf_reg': 0.40202111748653485, 'random_strength': 33.967005476522594, 'grow_policy': 'SymmetricTree', 'max_bin': 377, 'min_data_in_leaf': 9, 'bootstrap_type': 'Bernoulli', 'subsample': 0.34041860031304016}. Best is trial 47 with value: 0.5382262996941896.


0:	learn: 0.5010923	test: 0.4967235	best: 0.4967235 (0)	total: 22.8ms	remaining: 1m 54s
100:	learn: 0.5550135	test: 0.5249745	best: 0.5270133 (97)	total: 2.19s	remaining: 1m 46s
200:	learn: 0.5723804	test: 0.5206058	best: 0.5297801 (166)	total: 4.33s	remaining: 1m 43s
300:	learn: 0.5905847	test: 0.5289064	best: 0.5319645 (266)	total: 6.46s	remaining: 1m 40s
bestTest = 0.5319644677
bestIteration = 266
Shrink model to first 267 iterations.


[I 2022-01-24 11:42:07,095] Trial 48 finished with value: 0.5319644677442843 and parameters: {'learning_rate': 0.2991190718703116, 'depth': 5, 'l2_leaf_reg': 15.825376973648453, 'random_strength': 35.50502561620387, 'grow_policy': 'SymmetricTree', 'max_bin': 370, 'min_data_in_leaf': 7, 'bootstrap_type': 'Bayesian', 'bagging_temperature': 4.716095222953456}. Best is trial 47 with value: 0.5382262996941896.


0:	learn: 0.5045147	test: 0.5077909	best: 0.5077909 (0)	total: 18.1ms	remaining: 1m 30s
100:	learn: 0.5412874	test: 0.5213339	best: 0.5258483 (82)	total: 1.66s	remaining: 1m 20s
200:	learn: 0.5533023	test: 0.5232270	best: 0.5286151 (148)	total: 3.35s	remaining: 1m 19s
bestTest = 0.5286151158
bestIteration = 148
Shrink model to first 149 iterations.


[I 2022-01-24 11:42:13,102] Trial 49 finished with value: 0.5287607397699141 and parameters: {'learning_rate': 0.20457815367220397, 'depth': 3, 'l2_leaf_reg': 0.6537692485900841, 'random_strength': 48.45267534859222, 'grow_policy': 'Lossguide', 'max_bin': 344, 'min_data_in_leaf': 6, 'bootstrap_type': 'Bernoulli', 'subsample': 0.31034858192126935}. Best is trial 47 with value: 0.5382262996941896.


In [34]:
print(f'Best value (Accuracy): {cbred_study.best_value:.5f}')

cbred_best = cbred_study.best_params
print('Best params:')
for key, value in cbred_best.items():
    print(f'\t{key}: {value}')

Best value (Accuracy): 0.53823
Best params:
	learning_rate: 0.29547218456097296
	depth: 6
	l2_leaf_reg: 0.40202111748653485
	random_strength: 33.967005476522594
	grow_policy: SymmetricTree
	max_bin: 377
	min_data_in_leaf: 9
	bootstrap_type: Bernoulli
	subsample: 0.34041860031304016


In [35]:
model7 = CatBoostClassifier(**cb_base, **cbred_best)

In [36]:
oof_preds7, test_preds7 = custom_cross_val_predict(train_reduced, test, features, model7)

----------------------------------------
0:	learn: 0.5018204	test: 0.5022572	best: 0.5022572 (0)	total: 23.1ms	remaining: 1m 55s
100:	learn: 0.5916406	test: 0.5366244	best: 0.5376438 (92)	total: 2.15s	remaining: 1m 44s
200:	learn: 0.6387170	test: 0.5356051	best: 0.5405563 (164)	total: 4.37s	remaining: 1m 44s
bestTest = 0.5405562837
bestIteration = 164
Shrink model to first 165 iterations.
Fold #0: Accuracy = 0.54056     [Time: 7.52s]
----------------------------------------
0:	learn: 0.5017840	test: 0.5006553	best: 0.5006553 (0)	total: 22.8ms	remaining: 1m 53s
100:	learn: 0.5884366	test: 0.5248289	best: 0.5283239 (63)	total: 2.03s	remaining: 1m 38s
200:	learn: 0.6359863	test: 0.5257026	best: 0.5302170 (186)	total: 4.21s	remaining: 1m 40s
300:	learn: 0.6707930	test: 0.5296345	best: 0.5321101 (280)	total: 6.6s	remaining: 1m 43s
bestTest = 0.5321100917
bestIteration = 280
Shrink model to first 281 iterations.
Fold #1: Accuracy = 0.53211     [Time: 9.99s]
----------------------------------

In [37]:
sub7 = pd.read_csv(sub_url)
sub7['season'] = test_preds7.ravel()
sub7['season'] = sub7['season'].map(target_mapping)
sub7.to_csv('sub7.csv', index=False)

!head sub7.csv

season
a
a
u
a
a
u
a
u
u
